In [1]:
# download example data
# !pip install gdown
from downloaddata import download_example_data
download_example_data()


In [5]:
from wholeslidedata.iterators import create_batch_iterator
from wholeslidedata.accessories.asap.imagewriter import WholeSlideMaskWriter
from wholeslidedata.image.wholeslideimage import WholeSlideImage
import time
from pprint import pprint
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

In [7]:
from wholeslidedata.image.wholeslideimage import WholeSlideImage

In [8]:
image = WholeSlideImage('/tmp/TCGA-21-5784-01Z-00-DX1_tb_mask.tif', backend='asap')

In [9]:
image.spacings

[2.0217639732716486,
 4.043527946543297,
 8.087055893086594,
 16.181137798516215,
 32.390418208492044]

In [11]:
mode='training'
training_iterator = create_batch_iterator(mode=mode,
                                          user_config='./configs/slidingwindowconfig.yml',
                                          presets=('slidingwindow',),
                                          cpus=4, 
                                          number_of_batches=-1, 
                                          return_info=True)

spacing =0.5
tile_size = 1024
output_size = 1030
wsm_writer = WholeSlideMaskWriter()
with WholeSlideImage('/tmp/TCGA-21-5784-01Z-00-DX1.tif') as wsi:
    shape = wsi.shapes[wsi.get_level_from_spacing(spacing)]

wsm_writer.write(path='./test2.tif', spacing=spacing, dimensions=shape, tile_shape=(tile_size,tile_size))
for x_batch, y_batch, info in tqdm(training_iterator):
    point = info['sample_references'][0]['point']
    c, r = point.x-output_size//4, point.y-output_size//4
    wsm_writer.write_tile(tile=y_batch[0],coordinates=(int(c),int(r)), mask=y_batch[0])
wsm_writer.save()
training_iterator.stop()

Creating: ./test2.tif....
Spacing: 0.5
Dimensions: (36851, 25353)
Tile_shape: (1024, 1024)


  0%|          | 0/352 [00:00<?, ?it/s]